In [1]:
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from autorocks.viz import plots_setup
import torch
import gpytorch
import botorch.posteriors
from botorch import models
from botorch.optim import fit
import torch
import numpy as np
import pandas as pd
import botorch

output_location = "/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/Background/Figures/"




class Forrester(botorch.test_functions.SyntheticTestFunction):

    dim = 1
    _bounds = [(0.0, 1.0)]
    _optimal_value = -6.0
    _optimizers = [(0.78)]

    def evaluate_true(self, X: torch.Tensor) -> torch.Tensor:
        return torch.pow(6 * X - 2, 2) * torch.sin(12 * X - 4) 
        
    
problem = Forrester(negate = True)



In [3]:
num_observations = 5
train_x_full, train_y_full = plots_setup.generate_data(num_observations, problem)



In [4]:
model = models.FixedNoiseGP(
    train_X=train_x_full,
    train_Y=train_y_full,
    train_Yvar=torch.zeros_like(train_y_full)
)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
fit.fit_gpytorch_scipy(mll)


In [5]:
test_x = torch.linspace(0, 1)
test_y = problem(test_x)
cr = plots_setup.predict(model = model, test_x = test_x, observation_noise = False)

lower, upper, mean = cr.lower, cr.upper, cr.mean
sorted_x = np.argsort(test_x)

# plot the test_x as an x axis, then plot the mean prediction, fill the lower and upper
# then plot the observed x and y


In [21]:
plt.style.use("ggplot")
sns.set_theme(style="white", rc={"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper")  # , font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"
plt.rc("text", usetex=False)
plt.rc("xtick", labelsize="large")
plt.rc("ytick", labelsize="large")
plt.rc("axes", labelsize="large")
plt.rc("pdf", use14corefonts=True)

 # Initialize plot
f, ax = plt.subplots(1, 1, figsize=(4, 3))

# Plot training data as red stars
ax.plot(train_x_full.numpy(), train_y_full.numpy(), 'r*')
# Plot predictive means as blue line
ax.plot(test_x.numpy(), mean.numpy(), 'b')
# Shade between the lower and upper confidence bounds
ax.fill_between(test_x.numpy(), lower.squeeze().numpy(), upper.squeeze().numpy(), alpha=0.2)
ax.legend(['Observed Data', "Real Function", 'Mean', 'Confidence'])
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())

output_format = "png"
f.savefig(f"{output_location}/bo_loop_gp.{output_format}", bbox_inches="tight", format=f"{output_format}", dpi=300)


In [20]:
f, ax = plt.subplots(1, 1, figsize=(4, 3))

with torch.no_grad():
    ei = botorch.acquisition.ExpectedImprovement(model, best_f = train_y_full.min())(test_x.reshape(-1, 1, 1))

data = pd.DataFrame([
    {'AF': 'EI', 'f(x)': ei.numpy(), "x": test_x.numpy() },
])

plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())

ax = sns.lineplot(data.explode(column=["f(x)", "x"]), x="x", y="f(x)", ax=ax, palette = "hls")
ax.legend(['Expected Improvement(M(X))'])

output_format = "png"
f.savefig(f"{output_location}/bo_loop_acf.{output_format}", bbox_inches="tight", format=f"{output_format}", dpi=300)
